In [1]:

from s3_athena_api import *

env = "production"
site = 'a40ba08c-5e27-40d9-b798-7020ca6e4cae'
band = "5"

download_filename = get_ap_scan(env, site, band)

s3_output= s3://mist-production-athena/results/
download_local_path = ../../downloads/
SELECT * FROM "secorapp_production"."cv_ap_scans" where site='a40ba08c-5e27-40d9-b798-7020ca6e4cae' and band=5 and dt='2020-03-20';
    
{'QueryExecutionId': 'c2a7c425-3487-442a-8bef-d8e88c106db6', 'ResponseMetadata': {'RequestId': '01ee57e5-ba56-4808-acbb-03a2a4590029', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Fri, 20 Mar 2020 23:38:59 GMT', 'x-amzn-requestid': '01ee57e5-ba56-4808-acbb-03a2a4590029', 'content-length': '59', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
results/c2a7c425-3487-442a-8bef-d8e88c106db6.csv ../../downloads/c2a7c425-3487-442a-8bef-d8e88c106db6.csv
checking if key exist
checking if key exist
{'ResponseMetadata': {'RequestId': '873C39C54C3EA329', 'HostId': 'J0l1PoHdSAapafk5B/CbKu//XGODSTtgDHVOL0+ylAEGF7HtN49faSGJAeFvc8N+tGtxYimfa6E=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'J0l1PoHdSAapafk5B/CbKu//XGODSTtg

In [2]:
import pandas as pd

# filename = "../downloads/results/f92186ad-642b-4ec7-9d3b-7cd4aa33c492.csv"
df = pd.read_csv(download_filename)
df.head(10)

,ap,ap2,band,channel,in_site,org,rssi,site,tim,dt
0,5c5b352e13f6,5c5b35af4410,5,36,False,56de201d-e63b-4312-9858-40f4cfe35c7f,-66.0,a40ba08c-5e27-40d9-b798-7020ca6e4cae,2020-03-20 16:24:07.000,2020-03-20
1,5c5b352e13f6,5c5b352e12b1,5,64,False,56de201d-e63b-4312-9858-40f4cfe35c7f,-68.0,a40ba08c-5e27-40d9-b798-7020ca6e4cae,2020-03-20 16:24:07.000,2020-03-20
2,5c5b352e13f6,5c5b352e0f87,5,44,False,56de201d-e63b-4312-9858-40f4cfe35c7f,-69.0,a40ba08c-5e27-40d9-b798-7020ca6e4cae,2020-03-20 16:24:07.000,2020-03-20
3,5c5b352e13f6,5c5b352e1482,5,149,False,56de201d-e63b-4312-9858-40f4cfe35c7f,-63.0,a40ba08c-5e27-40d9-b798-7020ca6e4cae,2020-03-20 16:24:07.000,2020-03-20
4,5c5b352e13f6,5c5b352e11f8,5,161,False,56de201d-e63b-4312-9858-40f4cfe35c7f,-69.0,a40ba08c-5e27-40d9-b798-7020ca6e4cae,2020-03-20 16:24:07.000,2020-03-20
5,5c5b352e13f6,5c5b352e1487,5,44,False,56de201d-e63b-4312-9858-40f4cfe35c7f,-66.0,a40ba08c-5e27-40d9-b798-7020ca6e4cae,2020-03-20 16:24:07.000,2020-03-20
6,5c5b352e13f6,5c5b352e1180,5,157,False,56de201d-e63b-4312-9858-40f4cfe35c7f,-69.0,a40ba08c-5e27-40d9-b798-7020ca6e4cae,2020-03-20 16:24:07.000,2020-03-20
7,5c5b352e13f6,5c5b352e1220,5,36,False,56de201d-e63b-4312-9858-40f4cfe35c7f,-57.0,a40ba08c-5e27-40d9-b798-7020ca6e4cae,2020-03-20 16:24:07.000,2020-03-20
8,5c5b352e13f6,5c5b352e14eb,5,161,False,56de201d-e63b-4312-9858-40f4cfe35c7f,-68.0,a40ba08c-5e27-40d9-b798-7020ca6e4cae,2020-03-20 16:24:07.000,2020-03-20
9,5c5b352e13f6,5c5b352e11a3,5,40,False,56de201d-e63b-4312-9858-40f4cfe35c7f,-59.0,a40ba08c-5e27-40d9-b798-7020ca6e4cae,2020-03-20 16:24:07.000,2020-03-20


In [3]:
# import csv

# with open(download_filename) as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     line_count = 0
#     for row in csv_reader:
#         if line_count == 0:
#             print(f'Column names are {", ".join(row)}')
#             line_count += 1
#         else:
#             if line_count< 10:
#                  print(row)
#             line_count += 1
#     print(f'Processed {line_count} lines.')